In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [67]:
trip_data_january = pd.read_parquet('yellow_tripdata_2023-01.parquet')
number_of_columns = len(trip_data_january.columns)
print(f"Number of columns in the dataset: {number_of_columns}")

Number of columns in the dataset: 19


In [68]:
trip_data_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [69]:
trip_duration_january = trip_data_january["tpep_dropoff_datetime"] - trip_data_january["tpep_pickup_datetime"]
trip_duration_january_std = (trip_duration_january.dt.total_seconds() / 60).std()
print(f"Standard deviation of trip duration in January: {trip_duration_january_std:.2f} minutes")

Standard deviation of trip duration in January: 42.59 minutes


In [70]:
outliers = trip_data_january[(trip_duration_january.dt.total_seconds() < 60) | (trip_duration_january.dt.total_seconds() > 60 * 60)].index
tdj_before_drop_outlier_count = len(trip_data_january)
trip_data_january = trip_data_january.drop(outliers)
tdj_after_drop_outlier_count = len(trip_data_january)
print(f"Fraction of records left after dropping outliers: {tdj_after_drop_outlier_count / tdj_before_drop_outlier_count:.2%}")

Fraction of records left after dropping outliers: 98.12%


In [71]:
from sklearn.feature_extraction import DictVectorizer

features = trip_data_january[['PULocationID', 'DOLocationID']].astype(str)

dicts = features.to_dict(orient='records')

dv = DictVectorizer()
X = dv.fit_transform(dicts)

print(f"Feature matrix shape: {X.shape}")
print(f"Number of columns: {X.shape[1]}")

Feature matrix shape: (3009173, 515)
Number of columns: 515


In [72]:
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression

y = (trip_data_january["tpep_dropoff_datetime"] - trip_data_january["tpep_pickup_datetime"]).dt.total_seconds() / 60

model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

rmse = root_mean_squared_error(y, y_pred)
print(f"RMSE: {rmse:.2f} minutes")


RMSE: 7.65 minutes


In [73]:
trip_data_february = pd.read_parquet('yellow_tripdata_2023-02.parquet')
trip_duration_february = (trip_data_february["tpep_dropoff_datetime"] - trip_data_february["tpep_pickup_datetime"]).dt.total_seconds() / 60
outliers = trip_data_february[(trip_duration_february < 1) | (trip_duration_february > 60 )].index
trip_data_february = trip_data_february.drop(outliers)

features = trip_data_february[['PULocationID', 'DOLocationID']].astype(str)
dicts  = features.to_dict(orient='records')
X_val = dv.transform(dicts)
y_val = (trip_data_february["tpep_dropoff_datetime"] - trip_data_february["tpep_pickup_datetime"]).dt.total_seconds() / 60
y_pred = model.predict(X_val) 
rmse = root_mean_squared_error(y_val, y_pred)
print(f"RMSE on February data: {rmse:.2f} minutes")

RMSE on February data: 7.81 minutes
